In [1]:
#import API info
import os
from google.colab import userdata
os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')

In [2]:
#install packages
!pip install langchain chromadb faiss-cpu tiktoken wikipedia langchain-community google-generativeai==0.8.5 google-ai-generativelanguage==0.6.15

**Wikipedia Retriever**

In [3]:
#Import library
from langchain_community.retrievers import WikipediaRetriever

In [4]:
# Initialize the retriever (optional: set language and top_k)
retriever = WikipediaRetriever(language="en", top_k=2)

In [5]:
#Define query
query = "What is the living expense in Mumbai from middle class perspective?"

# Get relevant Wikipedia documents
doc = retriever.invoke(query)

In [6]:
doc

[Document(metadata={'title': 'Bhopal disaster', 'summary': "On 3 December 1984, over 500,000 people in the vicinity of the Union Carbide India Limited pesticide plant in Bhopal, Madhya Pradesh, India were exposed to the highly toxic gas methyl isocyanate, in what is considered the world's worst industrial disaster. A government affidavit in 2006 stated that the leak caused approximately 558,125 injuries, including 38,478 temporary partial injuries and 3,900 severely and permanently disabling injuries. Estimates vary on the death toll, with the official number of immediate deaths being 2,259. Others estimate that 8,000 died within two weeks of the incident occurring, and another 8,000 or more died from gas-related diseases. In 2008, the Government of Madhya Pradesh paid compensation to the family members of victims killed in the gas release, and to the injured victims.\nThe owner of the factory, Union Carbide India Limited (UCIL), was majority-owned by the Union Carbide Corporation (UCC

In [7]:
#Print retrieved content
for i, j in enumerate(doc):
  print(f"\n--->Results {i+1}<---")
  print(f"Content:\n{j.page_content}")



--->Results 1<---
Content:
On 3 December 1984, over 500,000 people in the vicinity of the Union Carbide India Limited pesticide plant in Bhopal, Madhya Pradesh, India were exposed to the highly toxic gas methyl isocyanate, in what is considered the world's worst industrial disaster. A government affidavit in 2006 stated that the leak caused approximately 558,125 injuries, including 38,478 temporary partial injuries and 3,900 severely and permanently disabling injuries. Estimates vary on the death toll, with the official number of immediate deaths being 2,259. Others estimate that 8,000 died within two weeks of the incident occurring, and another 8,000 or more died from gas-related diseases. In 2008, the Government of Madhya Pradesh paid compensation to the family members of victims killed in the gas release, and to the injured victims.
The owner of the factory, Union Carbide India Limited (UCIL), was majority-owned by the Union Carbide Corporation (UCC) of the United States, with Indi

**Vector Store Retriever**

In [8]:
from langchain_community.vectorstores import Chroma
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_core.documents import Document

In [9]:
# Step 1: Your source documents
documents = [
    Document(page_content="LangChain helps developers build LLM applications easily."),
    Document(page_content="Chroma is a vector database optimized for LLM-based search."),
    Document(page_content="Embeddings convert text into high-dimensional vectors."),
    Document(page_content="GoogleGenAI provides powerful embedding models."),
]

In [10]:
# Step 2: Initialize embedding model
embedding_model = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

# Step 3: Create Chroma vector store in memory
vector_store = Chroma.from_documents(
    documents = documents,
    embedding = embedding_model,
    collection_name= "my_collection"
    )

In [11]:
# Step 4: Convert vectorstore into a retriever
retriever = vector_store.as_retriever(search_kwargs = {"k":2})

In [12]:
query = "What is Chroma used for?"
results = retriever.invoke(query)

In [13]:
for i, doc in enumerate(results):
  print(f"\n--->Results {i+1}<---")
  print(f"Content:\n{doc.page_content}")


--->Results 1<---
Content:
Chroma is a vector database optimized for LLM-based search.

--->Results 2<---
Content:
Embeddings convert text into high-dimensional vectors.


**Maximal Marginal Relevance (MMR)**

In [14]:
# Sample documents
docs = [
    Document(page_content="LangChain makes it easy to work with LLMs."),
    Document(page_content="LangChain is used to build LLM based applications."),
    Document(page_content="Chroma is used to store and search document embeddings."),
    Document(page_content="Embeddings are vector representations of text."),
    Document(page_content="MMR helps you get diverse results when doing similarity search."),
    Document(page_content="LangChain supports Chroma, FAISS, Pinecone, and more."),
]

In [15]:
from langchain_community.vectorstores import FAISS

embedding_model = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

vector_store = FAISS.from_documents(
    documents = docs,
    embedding = embedding_model
)

In [16]:
retriever = vector_store.as_retriever(
    search_type="mmr", # <-- This enables MMR
    search_kwargs={"k": 3, "lambda_mult": 0.5}) # k = no. of top results, lambda_mult = relevance-diversity balance

In [17]:
query = "what is langchain?"
results = retriever.invoke(query)

In [18]:
for i, doc in enumerate(results):
  print(f"\n--->Results {i+1}<---")
  print(f"Content:\n{doc.page_content}")


--->Results 1<---
Content:
LangChain is used to build LLM based applications.

--->Results 2<---
Content:
Embeddings are vector representations of text.

--->Results 3<---
Content:
LangChain supports Chroma, FAISS, Pinecone, and more.


**Multiquery Retriever**

In [19]:
from langchain_community.vectorstores import FAISS
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_core.documents import Document
from langchain.retrievers.multi_query import MultiQueryRetriever

In [20]:
# Relevant health & wellness documents
all_docs = [
    Document(page_content="Regular walking boosts heart health and can reduce symptoms of depression.", metadata={"source": "H1"}),
    Document(page_content="Consuming leafy greens and fruits helps detox the body and improve longevity.", metadata={"source": "H2"}),
    Document(page_content="Deep sleep is crucial for cellular repair and emotional regulation.", metadata={"source": "H3"}),
    Document(page_content="Mindfulness and controlled breathing lower cortisol and improve mental clarity.", metadata={"source": "H4"}),
    Document(page_content="Drinking sufficient water throughout the day helps maintain metabolism and energy.", metadata={"source": "H5"}),
    Document(page_content="The solar energy system in modern homes helps balance electricity demand.", metadata={"source": "I1"}),
    Document(page_content="Python balances readability with power, making it a popular system design language.", metadata={"source": "I2"}),
    Document(page_content="Photosynthesis enables plants to produce energy by converting sunlight.", metadata={"source": "I3"}),
    Document(page_content="The 2022 FIFA World Cup was held in Qatar and drew global energy and excitement.", metadata={"source": "I4"}),
    Document(page_content="Black holes bend spacetime and store immense gravitational energy.", metadata={"source": "I5"}),
]

In [21]:
#Initialize embedding model and create vector store
embedding_model = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

vector_store = FAISS.from_documents(
    documents = all_docs,
    embedding = embedding_model
)

In [22]:
#create retrievers
similarity_retriever = vector_store.as_retriever(search_type="mmr", search_kwargs={"k": 5})

multiquery_retriever = MultiQueryRetriever.from_llm(
    retriever = vector_store.as_retriever(search_kwargs={"k":5}),
    llm = ChatGoogleGenerativeAI(model = "gemini-2.0-flash")
)

In [23]:
# Query
query = "How to improve energy levels and maintain balance?"

In [24]:
# Retrieve results
similarity_results = similarity_retriever.invoke(query)
multiquery_results= multiquery_retriever.invoke(query)

In [25]:
for i, doc in enumerate(similarity_results):
    print(f"\n--- Result {i+1} ---")
    print(doc.page_content)

print("*"*150)

for i, doc in enumerate(multiquery_results):
    print(f"\n--- Result {i+1} ---")
    print(doc.page_content)


--- Result 1 ---
Drinking sufficient water throughout the day helps maintain metabolism and energy.

--- Result 2 ---
The solar energy system in modern homes helps balance electricity demand.

--- Result 3 ---
Mindfulness and controlled breathing lower cortisol and improve mental clarity.

--- Result 4 ---
Python balances readability with power, making it a popular system design language.

--- Result 5 ---
The 2022 FIFA World Cup was held in Qatar and drew global energy and excitement.
******************************************************************************************************************************************************

--- Result 1 ---
Drinking sufficient water throughout the day helps maintain metabolism and energy.

--- Result 2 ---
Mindfulness and controlled breathing lower cortisol and improve mental clarity.

--- Result 3 ---
Consuming leafy greens and fruits helps detox the body and improve longevity.

--- Result 4 ---
Deep sleep is crucial for cellular repair an

**Contextual Compression Retriever**

In [39]:
from langchain_community.vectorstores import FAISS
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain_core.documents import Document

In [40]:
# Recreate the document objects from the previous data
docs = [
    Document(page_content=(
        """The Grand Canyon is one of the most visited natural wonders in the world.
        Photosynthesis is the process by which green plants convert sunlight into energy.
        Millions of tourists travel to see it every year. The rocks date back millions of years."""
    ), metadata={"source": "Doc1"}),

    Document(page_content=(
        """In medieval Europe, castles were built primarily for defense.
        The chlorophyll in plant cells captures sunlight during photosynthesis.
        Knights wore armor made of metal. Siege weapons were often used to breach castle walls."""
    ), metadata={"source": "Doc2"}),

    Document(page_content=(
        """Basketball was invented by Dr. James Naismith in the late 19th century.
        It was originally played with a soccer ball and peach baskets. NBA is now a global league."""
    ), metadata={"source": "Doc3"}),

    Document(page_content=(
        """The history of cinema began in the late 1800s. Silent films were the earliest form.
        Thomas Edison was among the pioneers. Photosynthesis does not occur in animal cells.
        Modern filmmaking involves complex CGI and sound design."""
    ), metadata={"source": "Doc4"})
]

In [41]:
#Initialize embedding model and create vector store
embedding_model = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

vector_store = FAISS.from_documents(docs, embedding_model)

In [42]:
base_retriever = vector_store.as_retriever(search_kwargs={"k": 5})

In [43]:
# Set up the compressor using an LLM
llm = ChatGoogleGenerativeAI(model = "gemini-2.0-flash")
compressor = LLMChainExtractor.from_llm(llm)

In [44]:
# Create the contextual compression retriever
compression_retriever = ContextualCompressionRetriever(
    base_retriever = base_retriever,
    base_compressor =compressor
)

In [49]:
# Query the retriever
query = "What is photosynthesis?"
compressed_results = compression_retriever.invoke(query)

In [50]:
for i, doc in enumerate(compressed_results):
    print(f"\n--- Result {i+1} ---")
    print(doc.page_content)


--- Result 1 ---
Photosynthesis does not occur in animal cells.

--- Result 2 ---
Photosynthesis is the process by which green plants convert sunlight into energy.

--- Result 3 ---
The chlorophyll in plant cells captures sunlight during photosynthesis.
